In [1]:
from getpass import getpass


# Recolher usuário e senha para uso do API Inventsys
usr = input('Informa usuário Inventsys: ')
pwd = getpass('Informa senha Inventsys : ')

Informa usuário Inventsys: guilherme
Informa senha Inventsys : ········


In [2]:
import requests
import json

# Usar o requests para fazer login no API e receber o token para outros comandos REST
url = 'https://api.inventsys.com.br/v4/login'
payload = "{\n  \"username\": \"" + usr + "\",\n  \"password\": \"" + pwd + "\"\n}"
headers = {
  'Account': 'stesa',
  'Content-Type': 'application/json',
    
}
r = requests.request('POST', url, headers = headers, data = payload, allow_redirects=False)

#Encontrar o item token dentro do json resposta ao request e depois opcionalmente plotá-lo
mytoken = r.json()['token']
print(mytoken)

65acee441c19ad1


In [3]:
#Enviar request GET para receber um json com todos os ativos do projeto escolhido
payload = {}
headers = {
    
  'Account': 'stesa',
  'Token': mytoken,
    
}

In [4]:
prjurl = 'https://api.inventsys.com.br/v4/projects'
projetos = requests.request('GET', prjurl, headers=headers, data=payload, allow_redirects=False)
#print(projetos.json()['projects'])

In [14]:

import ipywidgets as widgets
from ipywidgets import interact


variavelprj=""
listaprj=[]
for i in range(0,len(projetos.json()['projects'])):
    prj=(projetos.json()['projects'][i]['id'])
    nome=(projetos.json()['projects'][i]['name'])
    variavelprj=(str(prj))
    listaprj.append(variavelprj)

print('Informa ID do projeto no inventsys: ')

menuprj = widgets.Dropdown(
    options=listaprj,
    value='10762',
    description='Projeto:',
)


menuprj

botaoprojeto=widgets.Button(
    description='Submeter',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


outputprojeto = widgets.Output()

display(menuprj, botaoprojeto, outputprojeto)
outputprojeto
projectid=''
def on_button_clicked_prj(b):
    global projectid
    with outputprojeto:
        projectid = menuprj.value
        print('Projeto selecionado: '+projectid)
        


botaoprojeto.on_click(on_button_clicked_prj)

Informa ID do projeto no inventsys: 


Dropdown(description='Projeto:', index=12, options=('10954', '10956', '10961', '10916', '10753', '10724', '107…

Button(description='Submeter', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

In [16]:
caturl = 'https://api.inventsys.com.br/v4/projects/'+projectid+'/categories'
categorias = requests.request('GET', caturl, headers=headers, data=payload, allow_redirects=False)
#print(categorias.json()['categories'])

In [17]:
catvariavel=""
catlista=[]
for i in range(0,len(categorias.json()['categories'])):
    cat=(categorias.json()['categories'][i]['id'])
    nome=(categorias.json()['categories'][i]['name'])
    catvariavel=(str(cat))
    catlista.append(catvariavel)

print('Informa ID da categoria no inventsys: ')
menucat = widgets.Dropdown(
    options=catlista,
    value=catlista[0],
    description='Categoria:',
)


menucat

botaocat=widgets.Button(
    description='Submeter',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


outputcat = widgets.Output()

display(menucat, botaocat, outputcat)

categoryid=''
def on_button_clicked_cat(b):
    global categoryid
    with outputcat:
        categoryid = menucat.value
        print('Categoria selecionada: '+categoryid)

botaocat.on_click(on_button_clicked_cat)

Informa ID da categoria no inventsys: 


Dropdown(description='Categoria:', options=('18278', '18284', '20685', '20686'), value='18278')

Button(description='Submeter', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

In [10]:
page = 1
registros = []
while True:
    try:
        url = 'https://api.inventsys.com.br/v4/projects/'+projectid+'/items?category='+categoryid+'&pagesize=1&page='+str(page)
        ativos = requests.request('GET', url, headers=headers, data=payload, allow_redirects=False)
        
        
        
                
        #print(ativos.json()['items'])
        registros = registros + ativos.json()['items']
        
        #Se quiser ver os dados estruturados na resposta completa:
        #print(ativos.text)
        
        #Se quiser ver os headers:
        #print(ativos.headers)
    except HTTPError:
        # handle HTTPError
        logging.error('HTTPError')
    # ... put any other Exception you need to handle here
    except IndexError:
        break
    except Exception as e:
        # for handle unknown exception
        logging.error('Unknown exception')
    else:
        if len(ativos.json()['items'])==0:
            break
        else:
            page += 1

print(registros)

[{'id': 303985, 'item_id_parent': None, 'reference': ' REM/2', 'subreference1': 'Remoçao de Fauna', 'subreference2': ' BR-287, Cipriano de Oliveira Vera Cruz / BR', 'reference_alpha': None, 'reference_numeric': None, 'oid': 'REM/2', 'code': None, 'code_custom': None, 'name': None, 'image': 'https://static.inventsys.com.br/278/thumb/f-3309902-200x200c.jpg', 'situations': [], 'project_id': 10963, 'project': {'id': 10963, 'name': 'EGR Remoçao de Fauna', 'color': '#1abc9c'}, 'category_id': 20705, 'category': {'id': 20705, 'name': 'Remoçao de Fauna'}, 'area_id': None, 'area': None, 'location': {'lat': -29.700836181641, 'lng': -52.529865264893, 'address': {'region': 'RS', 'city': 'Vera Cruz', 'district': 'Cipriano de Oliveira', 'zipcode': None, 'street': 'BR-287', 'street_number': None, 'desc': None, 'full': 'BR-287, Cipriano de Oliveira Vera Cruz / BR'}}, 'event_last': None, 'description': None, 'search_terms': 'REM/2 EGR Remoçao de Fauna Remoçao de Fauna BR-287, Cipriano de Oliveira Vera C

In [56]:
#Escolha os campos que vao ser enviados para o Postgres

#Puxar os 50 records mais recentes
#urlptexto = 'https://api.inventsys.com.br/v4/projects/'+projectid+'/items?category='+categoryid+'&pagesize=1'
#ativosemtexto = requests.request('GET', urlptexto, headers=headers, data=payload, allow_redirects=True)              
#print(ativosemtexto.text)

In [29]:
import psycopg2
from psycopg2 import sql

#Abrir os trabalhos para trabalhar com Postgres. Criar conexão e cursor como base.
#conn = psycopg2.connect("host=localhost dbname=postgres user=guilhermeiablonovski")
#cur = conn.cursor()




#Define our connection string
conn_string = "host=localhost dbname=postgres user=guilhermeiablonovski"
 
# print the connection string we will use to connect
print('Connecting to database->%s' % (conn_string))
 
# get a connection, if a connect cannot be made an exception will be raised here
conn = psycopg2.connect(conn_string)
 
# conn.cursor will return a cursor object, you can use this cursor to perform queries
cur = conn.cursor()
print('Connected!\n')

Connecting to database->host=localhost dbname=postgres user=guilhermeiablonovski
Connected!



In [30]:
dropdbgenerica = """DROP TABLE IF EXISTS {}""" 


createdbgenerica = """CREATE UNLOGGED TABLE IF NOT EXISTS {}(
id integer PRIMARY KEY,
created_at DATE,
updated_at DATE,
name text,
image text,
project text,
category_id integer,
category_name text,
latitude real,
longitude real
);""" 

nometabela=projectid+'_'+categoryid



dbobracorrente = """DROP TABLE IF EXISTS egrfauna_obracorrente;
CREATE UNLOGGED TABLE IF NOT EXISTS egrfauna_obracorrente(
id integer PRIMARY KEY,
created_at DATE,
updated_at DATE,
name text,
image text,
project text,
category_id integer,
category_name text,
latitude real,
longitude real,
tipo text,
dimensao_passagem text,
grau_obstrucao integer,
natureza_obstrucao text
);""" 


dbobraespecial = """DROP TABLE IF EXISTS egrfauna_obraespecial;
CREATE UNLOGGED TABLE IF NOT EXISTS egrfauna_obraespecial(
id integer PRIMARY KEY,
created_at DATE,
updated_at DATE,
name text,
image text,
project text,
category_id integer,
category_name text,
latitude real,
longitude real,
tipo text,
largura_passagem real,
altura_passagem real,
margem_seca text,
grau_obstrucao text,
natureza_obstrucao text,
);""" 


dbarmadilhas = """DROP TABLE IF EXISTS egrfauna_armadilhas;
CREATE UNLOGGED TABLE IF NOT EXISTS egrfauna_armadilhas(
id integer PRIMARY KEY,
created_at DATE,
updated_at DATE,
name text,
image text,
project text,
category_id integer,
category_name text,
latitude real,
longitude real,
observacoes text,
instalacao DATE,
IDcartao text,
IDcamera text,
IDbueiro text,
estrada text,
foto_armadilha text,
gps_lat real,
gps_long real,
gps_alt real,
gps_acc real);""" 
    

dbatropelamentos = """DROP TABLE IF EXISTS egrfauna_atropelamentos;
CREATE UNLOGGED TABLE IF NOT EXISTS egrfauna_atropelamentos(
id integer PRIMARY KEY,
created_at DATE,
updated_at DATE,
name text,
image text,
project text,
category_id integer,
category_name text,
latitude real,
longitude real,
estrada text,
grupo text,
esp_mamifero text,
esp_ave text,
esp_reptil text,
esp_anfibio text,
esp_outro text,
idade text,
estado text,
posicao text,
id_etiqueta text,
nome_comum text,
sexo text,
observacoes text,
ponto_gps text
);"""



if categoryid=='20685':
    cur.execute(dbarmadilhas)
else:
    if categoryid=='20686':
        cur.execute(dbatropelamentos)
    else:
        if categoryid=='18278':
            cur.execute(dbobraespecial)
        else:
            if categoryid=='18284':
                cur.execute(dbobracorrente)
            else:
                cur.execute(sql.SQL(dropdbgenerica)
                            .format(sql.Identifier(nometabela)))
                cur.execute(sql.SQL(createdbgenerica)
                            .format(sql.Identifier(nometabela)))

conn.commit()

In [31]:
categoriasegr={'18284', '18278', '20685', '20686'}
    


if categoryid not in categoriasegr:
    for item in registros:
        genericfields=[
            item['id'],
            item['created_at'],
            item['updated_at'],
            item['name'],
            item['image'],
            item['project']['name'],
            item['category_id'],
            item['category']['name'],
            item['location']['lat'],
            item['location']['lng']
        ]
        my_data=[field for field in genericfields]
        cur.execute(sql.SQL("INSERT INTO {} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)").format(sql.Identifier(nometabela)),tuple(my_data))
else:
    print('.')
    
    
if categoryid=='18284':
        for item in registros:
            obracorrentefields = [
            item['id'],
            item['created_at'],
            item['updated_at'],
            item['name'],
            item['image'],
            item['project']['name'],
            item['category_id'],
            item['category']['name'],
            item['location']['lat'],
            item['location']['lng'],
            item['info'][0]['value'],
            item['info'][1]['value'],
            item['info'][2]['value'],
            item['info'][3]['value']
            ]
            my_data = [field for field in obracorrentefields]
            cur.execute("INSERT INTO egrfauna_obracorrente VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", tuple(my_data))
else:
    print('.')


if categoryid=='18278':
    for item in registros:
        obraespecialfields = [
        item['id'],
        item['created_at'],
        item['updated_at'],
        item['name'],
        item['image'],
        item['project']['name'],
        item['category_id'],
        item['category']['name'],
        item['location']['lat'],
        item['location']['lng'],
        item['info'][0]['value'],
        item['info'][1]['value'],
        item['info'][2]['value'],
        item['info'][3]['value'],
        item['info'][4]['value'],
        item['info'][5]['value']
        ]
        my_data = [field for field in obraespecialfields]
        cur.execute("INSERT INTO egrfauna_obraespecial VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", tuple(my_data))
else:
    print('.')



if categoryid=='20685':
    for item in registros:
        armadilhafields = [
        item['id'],
        item['created_at'],
        item['updated_at'],
        item['name'],
        item['image'],
        item['project']['name'],
        item['category_id'],
        item['category']['name'],
        item['location']['lat'],
        item['location']['lng'],
        item['info'][0]['value'],
        item['info'][1]['value'],
        item['info'][2]['value'],
        item['info'][3]['value'],
        item['info'][4]['value'],
        item['info'][5]['value'],
        item['info'][6]['value'],
        item['info'][7]['value'],
        item['info'][8]['value'],
        item['info'][9]['value'],
        item['info'][10]['value'],
        ]
        my_data = [field for field in armadilhafields]
        cur.execute("INSERT INTO egrfauna_armadilhas VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", tuple(my_data))
else:
        print('.')
    
if categoryid=='20686':
    for item in registros:
        atropelamentofields = [
        item['id'],
        item['created_at'],
        item['updated_at'],
        item['name'],
        item['image'],
        item['project']['name'],
        item['category_id'],
        item['category']['name'],
        item['location']['lat'],
        item['location']['lng'],
        item['info'][0]['value'],
        item['info'][1]['value'],
        item['info'][2]['value'],
        item['info'][3]['value'],
        item['info'][4]['value'],
        item['info'][5]['value'],
        item['info'][6]['value'],
        item['info'][7]['value'],
        item['info'][8]['value'],
        item['info'][9]['value'],
        item['info'][10]['value'],
        item['info'][11]['value'],
        item['info'][12]['value'],
        item['info'][13]['value'],
        item['info'][14]['value']
        ]
        my_data = [field for field in atropelamentofields]
        cur.execute("INSERT INTO egrfauna_atropelamentos VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", tuple(my_data))
else:
    print('.')
    
conn.commit()
    
    
    

.
.
.
.
